In [ ]:
%%sql
CREATE DATABASE pokerdata

In [ ]:
%%pyspark
df = spark.read.load('abfss://data@<storageacct>.dfs.core.windows.net/JSON/*/*/*/*.json', format='json')
df.write.mode("overwrite").saveAsTable("pokerdata.rawgames")

In [2]:
spark.sql("CREATE TABLE pokerdata.GameDetails  (GameId STRING, TableName string, GameTime string, GameActionId int, Player string, Streetname string, Action string, Amount float) USING PARQUET")

In [3]:
%%sql
Insert INTO pokerdata.gamedetails
SELECT  rg.id GameId
--, a.actions
    , rg.tablename TableName
    , rg.timestamp GameTime
    , a.actions.GameActionId
    , a.actions.actor Player
    , a.actions.streetname Streetname
    , a.actions.action Action
    , a.actions.amount Amount
FROM pokerdata.rawgames rg
JOIN (
    SELECT id, explode(actions) actions
    FROM pokerdata.rawgames 
) a ON rg.id = a.id
